In [25]:
import numpy as np
from scipy.ndimage import (gaussian_filter, sobel)

In [2]:
import bokeh
from ipywidgets import (interact)
from bokeh.plotting import (figure, show, output_notebook)
from bokeh.models import (LinearColorMapper, Ticker, ColorBar)
from bokeh.io import (push_notebook)

import os

output_notebook()

Loading BokehJS ...

In [74]:
x = np.linspace(-1.5, 1.5, 100)
y = x[None, :, None]
z = x[:, None, None]
x = x[None, None, :]

In [75]:
r = 4 * np.sqrt(x**2 + y**2) - 3
t = np.arctan2(x, y) / (np.pi * 2/3)

In [76]:
p = (r**2 + t**2) + (4*np.sqrt(x**2 + z**2) - 3)**2 * (z**2 + y**2)

In [77]:
def plot_cube(plot_data):
    n, m = plot_data.min(), plot_data.max()
    fig = figure(x_range=(0, plot_data.shape[1]), y_range=(0, plot_data.shape[2]), plot_width=400, plot_height=400)
    clm = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=n, high=m)
    img = fig.image(image=[plot_data[0,:,:]], x=0, y=0, dw=plot_data.shape[1], dh=plot_data.shape[2], color_mapper = clm)

    def update(y):  # , m):
        t = y # * 12 + m
        img.data_source.data['image'] = [plot_data[t]]
        push_notebook()

    show(fig, notebook_handle=True)
    interact(update, y=(0, plot_data.shape[0] - 1))
    
def plot_image(data):
    ratio = data.shape[0] / data.shape[1]
    p = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=int(378/ratio), plot_height=378)
    p.image(image=[data], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.viridis(255))
    show(p)

In [82]:
q = np.where(p < 1, 1.0, 0.0)
smooth_q = gaussian_filter(q, 2.0).astype('float32')
plot_image(smooth_q[:, 30, :])

In [83]:
def full_sobel(smooth_data):
    dim = len(smooth_data.shape)
    sb = np.array([
        sobel(smooth_data, mode=['reflect', 'reflect', 'wrap'], axis=i)
        for i in range(dim)])
    sb = np.r_[sb, np.ones_like(sb[0:1])]
    norm = np.sqrt((sb[:dim]**2).sum(axis=0))
    sb /= norm
    
    return sb

In [84]:
sb = full_sobel(smooth_q)

/home/johannes/.local/share/workon/abrupt/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
/home/johannes/.local/share/workon/abrupt/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [85]:
from hyper_canny import (cp_double_threshold, cp_edge_thinning)

In [90]:
dat = sb.transpose([3,2,1,0]).astype('float32')
mask = cp_edge_thinning(dat)
edges = cp_double_threshold(dat, mask, 1./1, 1./0.5)

In [92]:
plot_cube(edges)

interactive(children=(IntSlider(value=49, description='y', max=99), Output()), _dom_classes=('widget-interact',))